In [ ]:
#put in parent directory

In [11]:
import os
import pandas as pd
import numpy as np
from openpyxl.styles import Alignment

# Function to get all directories starting with 'GA'
def get_ga_directories(parent_folder):
    return [d for d in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, d)) and d.startswith('GA')]

# Set the parent folder where all GA directories are located
parent_folder = '.'  # Change this to the actual path if needed

# Get the list of top-level GA directories
ga_directories = get_ga_directories(parent_folder)

def try_convert_numeric(val):
    try:
        return pd.to_numeric(val)
    except ValueError:
        return val

# Process each GA directory
for ga_dir in ga_directories:
    ga_path = os.path.join(parent_folder, ga_dir)
    excel_file = os.path.join(ga_path, f'{ga_dir}.xlsx')
    
    # Get all subdirectories in the GA directory
    subdirectories = [d for d in os.listdir(ga_path) if os.path.isdir(os.path.join(ga_path, d)) and d.startswith(ga_dir)]
    
    # Initialize an Excel writer for this GA
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        # Process each subdirectory (experiment)
        for subdir in subdirectories:
            combined_df = pd.DataFrame()
            
            # Loop over each file in the subdirectory
            for i in range(1, 11):
                file_name = os.path.join(ga_path, subdir, f'{subdir}-Result{i}.txt')
                column_name = f'{i}'
                
                if os.path.isfile(file_name):
                    try:
                        with open(file_name, 'r') as file:
                            content = file.read().splitlines()
                        
                        # Create a DataFrame and attempt to convert to numeric
                        df = pd.DataFrame({column_name: content})
                        df[column_name] = df[column_name].apply(try_convert_numeric)
                        
                        combined_df = pd.concat([combined_df, df], axis=1)
                    except Exception as e:
                        print(f"Error reading file {file_name}: {e}")
                        continue
                else:
                    print(f"File not found: {file_name}")
                    continue
            
            if not combined_df.empty:
                try:
                    # Ensure numeric columns are written as numbers
                    combined_df = combined_df.apply(pd.to_numeric)
                    
                    # Create the 'Generation \ Experiment' column
                    gen_exp_column = list(range(1, 2001))  # First 2000 rows
                    gen_exp_column.append('')  # Skip 1 line
                    gen_exp_column.append('Values')  # Add 'Values'
                    gen_exp_column.extend(range(1, 31))  # Add 1 to 30
                    gen_exp_column.append('')  # Skip 1 line
                    gen_exp_column.append('Time')  # Add 'Time'
                    gen_exp_column.append('')  # Skip 1 line
                    
                    # Create a new DataFrame with the correct number of rows
                    new_df = pd.DataFrame(index=range(len(gen_exp_column)))
                    
                    # Add the 'Generation \ Experiment' column
                    new_df['Generation \\ Experiment'] = gen_exp_column
                    
                    # Add data from combined_df, filling with empty strings where necessary
                    for col in combined_df.columns:
                        new_df[col] = combined_df[col].tolist() + [''] * (len(new_df) - len(combined_df))
                    
                    # Write to Excel
                    new_df.to_excel(writer, sheet_name=subdir, index=False)
                    
                    # Get the worksheet
                    worksheet = writer.sheets[subdir]
                    
                    # Adjust column widths and alignment
                    for idx, col in enumerate(new_df.columns):
                        max_len = max(new_df[col].astype(str).map(len).max(), len(str(col)))
                        worksheet.column_dimensions[chr(65 + idx)].width = max_len + 2
                        
                        # Center align all columns except the first one
                        if idx > 0:
                            for cell in worksheet[chr(65 + idx)]:
                                cell.alignment = Alignment(horizontal='center')
                    
                except Exception as e:
                    print(f"Error writing to Excel sheet {subdir}: {e}")
            else:
                print(f"No files found for directory: {subdir}")
    
    print(f"Created Excel file: {excel_file}")

print("Done processing all GA directories.")

Created Excel file: .\GA1\GA1.xlsx
Created Excel file: .\GA10\GA10.xlsx
Created Excel file: .\GA11\GA11.xlsx
Created Excel file: .\GA12\GA12.xlsx
Created Excel file: .\GA13\GA13.xlsx
Created Excel file: .\GA14\GA14.xlsx
Created Excel file: .\GA15\GA15.xlsx
Created Excel file: .\GA16\GA16.xlsx
Created Excel file: .\GA2\GA2.xlsx
Created Excel file: .\GA3\GA3.xlsx
Created Excel file: .\GA4\GA4.xlsx
Created Excel file: .\GA5\GA5.xlsx
Created Excel file: .\GA6\GA6.xlsx
Created Excel file: .\GA7\GA7.xlsx
Created Excel file: .\GA8\GA8.xlsx
Created Excel file: .\GA9\GA9.xlsx
Done processing all GA directories.


In [ ]:
#v3